In [6]:
import numpy as np
from moabb.datasets import BNCI2014_001
from moabb.paradigms import MotorImagery, LeftRightImagery
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from mne.decoding import CSP
from moabb.evaluations import CrossSubjectEvaluation
from sklearn.pipeline import make_pipeline
from scipy import signal
from scipy.io import loadmat
import os
import mne
import numpy as np
from pyriemann.utils.mean import mean_riemann
from pyriemann.utils.distance import distance_riemann
from sklearn.model_selection import KFold
from tqdm import tqdm

In [7]:
import os
import numpy as np
import mne
from scipy.signal import butter, lfilter

# Define a causal bandpass filter function using a Butterworth design.
def causal_bandpass_filter(data, lowcut=8, highcut=30, fs=250, order=50):
    nyq = 0.5 * fs
    # Normalize the cutoff frequencies (Matlab's fir1 expects normalized cutoff frequencies
    low = lowcut / nyq
    high = highcut / nyq
    # Design the FIR filter. Note: order+1 coefficients are returned to match Matlab's fir1 which returns n+1 taps.
    b = signal.firwin(order + 1, [low, high], window='hamming', pass_zero=False)
    # Apply the filter causally using lfilter (this introduces a constant delay).
    filtered_data = signal.lfilter(b, [1.0], data)
    return filtered_data

In [ ]:

# Set the data directory where your GDF files are stored.
data_dir = '/home/vishwa/eeg_tl/Recreating papers/BCICIV_2a'  # Replace with your actual path

In [9]:
active_all_event_ids = {'769': 769, '770': 770, '771': 771, '772': 772}
active_lr_event_ids = {'769': 769, '770': 770}
unknown_event_id = {'783': 783} 

In [10]:
# With a sampling frequency of 250 Hz, 1001 samples equate to 1001/250 seconds.
sfreq = 250
tmin = 0.5       # Epoch start at cue onset.
# Set tmax so that n_samples = (tmax-tmin)*sfreq + 1 = 1001, i.e. 4 seconds long.
tmax = 2.5  # This gives 4.0 seconds.

In [ ]:
train_active_X = []         # List to hold numpy arrays with shape (n_trials, 22, 1001) per subject.
train_active_y = []         # List to hold event labels per subject.
train_active_metadata = []  # List to hold event metadata per subject.

# Loop over subjects. Assume files are named "A01T.gdf", "A02T.gdf", ..., "A09T.gdf".
for subj in range(1, 10):
    filename = os.path.join(data_dir, f'A{subj:02d}T.gdf')
    
    # Read the GDF file (using preload=True to load data into memory).
    train_raw = mne.io.read_raw_gdf(filename, preload=True, verbose=False, eog=['EOG-left', 'EOG-central', 'EOG-right'])
    # Retain only EEG channels (22 channels) and exclude EOG channels.
    train_raw.pick_types(eeg=True, eog=False)
    
    # Extract events corresponding only to the four desired types.
    train_active_events, _ = mne.events_from_annotations(train_raw, event_id=active_all_event_ids)
    
    # Create epochs from tmin to tmax.
    train_active_epochs = mne.Epochs(train_raw, train_active_events, event_id=active_all_event_ids, tmin=tmin, tmax=tmax,
                        baseline=None, preload=True, verbose=False)
    
    # Get the epoch data (num_epochs x 22 channels x 1001 samples).
    train_active_data = train_active_epochs.get_data()
    
    # Print the number of extracted epochs to verify
    print(f"Subject {subj}: Epoch data shape {train_active_data.shape}")
    
    # Sampling frequency from raw.info (should be 250).
    fs = int(train_raw.info['sfreq'])
    # print(fs)
    n_trials, n_channels, n_times = train_active_data.shape
    train_active_filtered_data = np.empty_like(train_active_data)
    
    # Apply the causal bandpass filter channel‐wise for each trial.
    for trial in range(n_trials):
        for ch in range(n_channels):
            train_active_filtered_data[trial, ch, :] = causal_bandpass_filter(
                train_active_data[trial, ch, :],
                lowcut=8,    # Lower bound of sensorimotor rhythm.
                highcut=30,  # Upper bound of sensorimotor rhythm.
                fs=fs,
                order=50     # Lower order for a smoother causal filter.
            )
    
    # Append the processed data, labels, and event metadata.
    train_active_X.append(train_active_filtered_data)
    train_active_y.append(train_active_epochs.events[:, 2])  # The third column holds the event code.
    train_active_metadata.append(train_active_epochs.events)

print("Loaded data for", len(train_active_X), "subjects.")

/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/contextlib.py:142: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: ['769', '770', '771', '772']
Subject 1: Epoch data shape (288, 22, 501)


/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/contextlib.py:142: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: ['769', '770', '771', '772']
Subject 2: Epoch data shape (288, 22, 501)


/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/contextlib.py:142: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


...

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: ['769', '770', '771', '772']
Subject 7: Epoch data shape (288, 22, 501)


/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/contextlib.py:142: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: ['769', '770', '771', '772']
Subject 8: Epoch data shape (288, 22, 501)


/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/contextlib.py:142: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: ['769', '770', '771', '772']
Subject 9: Epoch data shape (288, 22, 501)
Loaded data for 9 subjects.


In [ ]:
eval_active_X = []         
eval_active_y = []        
eval_active_metadata = [] 

# Loop over subjects. Assume files are named "A01T.gdf", "A02T.gdf", ..., "A09T.gdf".
for subj in range(1, 10):
    filename = os.path.join(data_dir, f'A{subj:02d}E.gdf')
    mat_data = loadmat(f'/home/vishwa/eeg_tl/Recreating papers/BCICIV_2A true labels/A{subj:02d}E.mat')
    true_y =  np.array(mat_data['classlabel'], dtype=np.int64).reshape(288,) + 768
    # Read the GDF file (using preload=True to load data into memory).
    eval_raw = mne.io.read_raw_gdf(filename, preload=True, verbose=False, eog=['EOG-left', 'EOG-central', 'EOG-right'])
    # Retain only EEG channels (22 channels) and exclude EOG channels.
    eval_raw.pick_types(eeg=True, eog=False)
    
    # Extract events corresponding only to the four desired types.
    eval_active_events, _ = mne.events_from_annotations(eval_raw, event_id=unknown_event_id)
    
    # Create epochs from tmin to tmax.
    eval_active_epochs = mne.Epochs(eval_raw, eval_active_events, event_id=unknown_event_id, tmin=tmin, tmax=tmax,
                        baseline=None, preload=True, verbose=False)
    
    # Get the epoch data (num_epochs x 22 channels x 1001 samples).
    eval_active_data = eval_active_epochs.get_data()
    
    # Print the number of extracted epochs to verify
    print(f"Subject {subj}: Epoch data shape {eval_active_data.shape}")
    
    # Sampling frequency from raw.info (should be 250).
    fs = int(eval_raw.info['sfreq'])
    # print(fs)
    n_trials, n_channels, n_times = eval_active_data.shape
    eval_active_filtered_data = np.empty_like(eval_active_data)
    
    # Apply the causal bandpass filter channel‐wise for each trial.
    for trial in range(n_trials):
        for ch in range(n_channels):
            eval_active_filtered_data[trial, ch, :] = causal_bandpass_filter(
                eval_active_data[trial, ch, :],
                lowcut=8,    # Lower bound of sensorimotor rhythm.
                highcut=30,  # Upper bound of sensorimotor rhythm.
                fs=fs,
                order=50     # Lower order for a smoother causal filter.
            )
    
    # Append the processed data, labels, and event metadata.
    eval_active_X.append(eval_active_filtered_data)
    eval_active_y.append(true_y)  # The third column holds the event code.
    eval_active_metadata.append(eval_active_epochs.events)

print("Loaded data for", len(eval_active_X), "subjects.")

/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/contextlib.py:142: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: ['783']
Subject 1: Epoch data shape (288, 22, 501)


/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/contextlib.py:142: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: ['783']
Subject 2: Epoch data shape (288, 22, 501)


/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/contextlib.py:142: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


...

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: ['783']
Subject 7: Epoch data shape (288, 22, 501)


/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/contextlib.py:142: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: ['783']
Subject 8: Epoch data shape (288, 22, 501)


/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/contextlib.py:142: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: ['783']
Subject 9: Epoch data shape (288, 22, 501)
Loaded data for 9 subjects.


In [13]:
len(train_active_X)

9

In [14]:
train_active_X[0].shape

(288, 22, 501)

In [15]:
print(len(train_active_y))

9


In [16]:
train_active_y[0].shape

(288,)

In [17]:
# Assuming you have similar variables for eval data:
# eval_active_X and eval_active_y

# Initialize lists to store concatenated data
concatenated_X = []
concatenated_y = []

# Loop through each subject
for subject in range(9):
    # Concatenate X data for current subject
    subject_X = np.concatenate([
        train_active_X[subject],
        eval_active_X[subject]
    ], axis=0)
    
    # Concatenate y data for current subject
    subject_y = np.concatenate([
        train_active_y[subject],
        eval_active_y[subject]
    ], axis=0)
    
    # Append to lists
    concatenated_X.append(subject_X)
    concatenated_y.append(subject_y)

# Convert lists to arrays if needed
concatenated_X = np.array(concatenated_X)
concatenated_y = np.array(concatenated_y)


In [18]:
def cov(X):
    covs = np.array([trial @ trial.T for trial in X])
    return covs

In [19]:
covs = cov(concatenated_X[0])

In [20]:
print(covs.shape)

(576, 22, 22)


In [21]:
covs_labels = concatenated_y[0]

In [22]:
print(covs_labels.shape)

(576,)


In [23]:
np.unique(covs_labels)

array([769, 770, 771, 772])

In [24]:
def minimum_distance_classifier(train_covs, train_labels, test_cov):
    """
    Implementation of Minimum Distance to Riemannian Mean classifier
    """
    classes = np.unique(train_labels)
    distances = np.zeros(len(classes))
    
    # Compute class-wise Riemannian means
    class_means = {}
    for i, k in enumerate(classes):  # Use enumerate to get index
        idx = train_labels == k
        class_means[k] = mean_riemann(train_covs[idx])
    
    # Compute distances to each class mean
    for i, k in enumerate(classes):  # Use enumerate to get index
        distances[i] = distance_riemann(test_cov, class_means[k])
    
    return classes[np.argmin(distances)]


In [25]:

def cross_validate_subject(subject_data, subject_labels, n_folds=30):
    """
    Perform 30-fold cross validation for a single subject
    
    Parameters:
    subject_data: array of shape (288, 22, 1001)
    subject_labels: array of shape (288,)
    """
    # First compute covariance matrices for all trials
    covs = cov(subject_data)
    # Initialize K-fold
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    # Store accuracies for each fold
    fold_accuracies = []
    
    # Perform cross validation
    for fold_idx, (train_idx, test_idx) in tqdm(enumerate(kf.split(covs))):
        # Split data into train and test
        train_covs = covs[train_idx]
        train_labels = subject_labels[train_idx]
        test_covs = covs[test_idx]
        test_labels = subject_labels[test_idx]
        
        # Predict each test trial
        predictions = []
        for test_cov in test_covs:
            pred = minimum_distance_classifier(train_covs, train_labels, test_cov)
            predictions.append(pred)
        
        # Calculate accuracy for this fold
        accuracy = np.mean(np.array(predictions) == test_labels)
        fold_accuracies.append(accuracy)
    
    return fold_accuracies


In [26]:
# subject_accuracies = cross_validate_subject(covs, covs_labels)
# print(np.mean(subject_accuracies))

In [27]:

# Run for all subjects
all_subjects_results = []
for subject in range(9):  # 9 subjects
    subject_accuracies = cross_validate_subject(
        concatenated_X[subject], 
        concatenated_y[subject]
    )
    all_subjects_results.append(subject_accuracies)

# Calculate overall statistics
all_subjects_results = np.array(all_subjects_results)
mean_accuracy = np.mean(all_subjects_results)
std_accuracy = np.std(all_subjects_results)

print(f"Mean accuracy across all subjects: {mean_accuracy:.3f} ± {std_accuracy:.3f}")


0it [00:00, ?it/s]

30it [03:15,  6.52s/it]
30it [03:40,  7.34s/it]
30it [03:26,  6.88s/it]
30it [03:31,  7.06s/it]
30it [04:56,  9.87s/it]
30it [04:33,  9.13s/it]
30it [04:28,  8.95s/it]
30it [04:28,  8.94s/it]
30it [04:05,  8.19s/it]

Mean accuracy across all subjects: 0.570 ± 0.189


In [28]:
all_subjects_results.shape

(9, 30)

In [29]:
for i in range(len(all_subjects_results)):
    print(f"accuracy_score of subj{i+1}: {np.mean(all_subjects_results[i]):.2f}")
print(f"Mean accuracy from data: {mean_accuracy:.2f} ± {std_accuracy:.2f}")

accuracy_score of subj1: 0.65
accuracy_score of subj2: 0.39
accuracy_score of subj3: 0.74
accuracy_score of subj4: 0.47
accuracy_score of subj5: 0.32
accuracy_score of subj6: 0.42
accuracy_score of subj7: 0.65
accuracy_score of subj8: 0.74
accuracy_score of subj9: 0.74
Mean accuracy from data: 0.57 ± 0.19
